# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
print(len(full_data_rows_list))
print(full_data_rows_list[:10])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Rokia TraorÃ\x83Â©', 'Logged In', 'Stefany', 'F', '0', 'White', '274.88608', 'free', 'Lubbock, TX', 'PUT', 'NextSong', '1.54071E+12', '693', 'Zen', '200', '1.54251E+12', '83'], ['', 'Logged In', 'Tucker', 'M', '0', 'Garrison', '', 'free', 'Oxnard-Thousand Oaks-Ventura, CA', 'GET', 'Home', '1.54083E+12', '555', '', '200', '1.54251E+12', '40'], ['Camila', 'Logged In', 'Tucker', 'M', '1', 'Garrison', '230.81751', 'free', 'Oxnard-Thousand Oaks-Ventura, CA', 'PUT', 'NextSong', '1.54083E+12', '555', 'Abrazame  (Version Acustica)', '200', '1.54251E+12', '40'], ['', 'Logged In', 'Ann', 'F', '0', 'Banks', '', 'free', 'Salt Lake City, UT', 'GET', 'Home', '1.5409E+12', '573', '', '200', '1.54253E+12', '99'], ['Carl Thomas', 'Logged In', 'Tucker', 'M', '0', 'Garrison', '196.67546', 'free', 'Oxnard-Thousand Oaks-Ventura, CA', 'PUT', 'NextSong', '1.54083E+12', '698', "You Ain't Right (Album Version)", '200', '1.54253E+12', '40'], ['N.E.R.D.', 'Logged In', 'James', 'M', '0', 'Martin', '242.99

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS my_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('my_keyspace')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




<img src="images/image_event_datafile_new.jpg">

#### Q1 - Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
# sessionId and itemInSession would be partition key here.
create_query = '''
CREATE TABLE IF NOT EXISTS songplays_song_info
(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))
'''

try:
    session.execute(create_query)
except Exception as e:
    print(e)          

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_song_info (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
q1_result = []
select_query = '''
SELECT artist, song, length
FROM songplays_song_info
WHERE sessionId = 338
AND itemInSession = 4
'''
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row)
    q1_result.append(row)

pd.DataFrame(q1_result)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Q2 - Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# set userId and sessionId to partition keys and itemInSession as clustering key
create_query = '''
CREATE TABLE IF NOT EXISTS songplays_session 
(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, 
 PRIMARY KEY ((userId, sessionId), itemInSession))
'''

try:
    session.execute(create_query)
except Exception as e:
    print(e)               

In [12]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [13]:
q2_result = []

select_query = '''
SELECT artist, song, firstName, lastName
FROM songplays_session 
WHERE userId = 10
AND sessionId = 182
'''
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
    q2_result.append(row)

pd.DataFrame(q2_result)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


,artist,song,firstname,lastname
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Q3 - Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [16]:
# Data Model: As sessionId and itemInSession together will be unique in the data, so here we still need to keep them
# as clustering key. Along with song as partition key. 

create_query = '''
CREATE TABLE IF NOT EXISTS songplays_song_user
(song text, userId int, firstName text, lastName text, 
 PRIMARY KEY ((song), userId))
'''

try:
    session.execute(create_query)
except Exception as e:
    print(e) 

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songplays_song_user (song, userId, firstName, lastName)"
        query = query + " VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [18]:
q3_result = []

select_query = '''
SELECT firstName, lastName 
FROM songplays_song_user
WHERE song = 'All Hands Against His Own'
'''
try:
    rows = session.execute(select_query)
except Exception as e:
    print(e)

print("Here are list of user who listened 'All Hands Against His Own'")

for row in rows:
    q3_result.append(row)
    print(row)

pd.DataFrame(q3_result)

Here are list of user who listened 'All Hands Against His Own'
Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [19]:
drop_query = ['drop table songplays_{}'.format(x) for x in ('song_info','session','song_user')]

In [20]:
for query in drop_query:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()